In [2]:
import torchvision.transforms.functional
from matplotlib import pyplot as plt
import torch
import numpy as np
from functools import partial
import torchvision.transforms as transforms
import os
from dipy.io import read_bvals_bvecs
from dipy.io.image import load_nifti
import torch.nn.functional as F
from tqdm import tqdm
import torchvision.transforms as T

In [ ]:
import os
import numpy as np
from dipy.io.image import load_nifti
from nibabel import Nifti1Image
from dipy.io import read_bvals_bvecs
import shutil
root = '/mnt/HCP_3T_251/'
log_path = os.path.join(root, 'problematic_subjects.log')

# Ensure log file is empty at the start
with open(log_path, 'w') as log_file:
    log_file.write('')

files = sorted(os.listdir(root))

for file in files:
    file_str = str(file)
    file_name = file_str.split("_")[0]
    print(f"subj {file_name} starts")
    data_path = os.path.join(root, file_name, 'T1w', 'Diffusion', 'data.nii.gz')
    bval_path = os.path.join(root, file_name, 'T1w', 'Diffusion', 'bvals')
    bvec_path = os.path.join(root, file_name, 'T1w', 'Diffusion', 'bvecs')
    mask_path = os.path.join(root, file_name, 'T1w', 'Diffusion', 'nodif_brain_mask.nii.gz')
    save_path_90 = os.path.join(root, file_name, 'diffusion_90_directions')
    save_path_180 = os.path.join(root, file_name, 'diffusion_180_directions')
    save_path_270 = os.path.join(root, file_name, 'diffusion_270_directions')
    os.makedirs(save_path_90, exist_ok=True)
    os.makedirs(save_path_180, exist_ok=True)
    os.makedirs(save_path_270, exist_ok=True)


    try:
        data, affine = load_nifti(data_path)
        bvals, bvecs = read_bvals_bvecs(bval_path, bvec_path)

        if data.shape[-1] != 288:
            with open(log_path, 'a') as log_file:
                log_file.write(f"{file_name} has incorrect data shape: {data.shape}\n")
            continue

        b0, b1, b2, b3 = [], [], [], []
        for i in range(288):
            b = bvals[i]
            if abs(b - 0) == min(abs(b - 0), abs(b - 1000), abs(b - 2000), abs(b - 3000)):
                b0.append(i)
            elif abs(b - 1000) == min(abs(b - 0), abs(b - 1000), abs(b - 2000), abs(b - 3000)):
                b1.append(i)
            elif abs(b - 2000) == min(abs(b - 0), abs(b - 1000), abs(b - 2000), abs(b - 3000)):
                b2.append(i)
            else:
                b3.append(i)

        b0_data = data[:, :, :, b0]
        
        b1_data = data[:, :, :, b1]
        b1_bvals = bvals[b1]
        b1_bvecs = bvecs[b1, :]
                
        # b2_data = data[:, :, :, b2]
        # b2_bvals = bvals[b2]
        # b2_bvecs = bvecs[b2, :]
        
        # b3_data = data[:, :, :, b3]
        # b3_bvals = bvals[b3]
        # b3_bvecs = bvecs[b3, :]

        # save shell data nii file
        print(f"subj {file_name} is being processed")
        GT_mask, _ = load_nifti(mask_path)
        average_b0 = np.mean(b0_data, axis=3)
        b0 = np.expand_dims(average_b0, 3)
        data_gt = b1_data / (b0 + 1e-5)
        data_gt = np.nan_to_num(data_gt, copy=True, nan=0, posinf=0, neginf=0)
        data_gt = np.clip(data_gt, 0, 1)
        data_gt *= np.expand_dims(GT_mask, axis=-1)
        print(f"subj {file_name} is saving")
        new_image = Nifti1Image(data_gt, np.eye(4))
        Nifti1Image(data_gt, affine).to_filename(os.path.join(save_path_90, f'{file_name}_90.nii.gz'))
        Nifti1Image(GT_mask, affine).to_filename(os.path.join(save_path_90, f'{file_name}_brain_mask.nii.gz'))

        np.savetxt(os.path.join(save_path_90, f'{file_name}_bvecs_90.txt'), b1_bvecs, delimiter=' ')
        np.savetxt(os.path.join(save_path_90, f'{file_name}_bvals_90.txt'), b1_bvals, delimiter=' ')
        # np.savetxt(os.path.join(save_path_180, f'{file_name}_bvecs_180.txt'), b2_bvecs, delimiter=' ')
        # np.savetxt(os.path.join(save_path_180, f'{file_name}_bvals_180.txt'), b2_bvals, delimiter=' ')
        # np.savetxt(os.path.join(save_path_270, f'{file_name}_bvecs_270.txt'), b3_bvecs, delimiter=' ')
        # np.savetxt(os.path.join(save_path_270, f'{file_name}_bvals_270.txt'), b3_bvals, delimiter=' ')

        # print(f"subj {file_name} is done")

    except Exception as e:
        with open(log_path, 'a') as log_file:
            log_file.write(f"{file_name} processing failed: {str(e)}\n")


In [ ]:
# —————————————————————————————————————————————————————————————————————————————————————————————————————————————
# |                              Generate 90 directions from 288 directions P2S denoised subjects             |     
# —————————————————————————————————————————————————————————————————————————————————————————————————————————————
root = '/mnt/disk4/hcp_fa_ad/hcp-78/'
save_path = '/mnt/disk1/munan/hcp_single/'
noddi_path = '/mnt/disk4/hcp_fa_ad/NODDI_LABELS/'
normalise = True
os.makedirs(save_path, exist_ok=True)
# sort files
files = sorted(os.listdir(root))
# index_1 = [7, 37, 39, 68, 69, 81]
# index_1_random = [4, 7, 17, 37, 45, 52, 58 ,68, 69, 81]
index_1 = [4, 7, 17, 37, 45, 52]
index_1_30 =[1, 4, 7, 10, 15, 17, 20, 25, 27, 30, 37, 39, 43, 45, 48, 50, 52, 54, 56, 58, 60, 68, 69, 72, 75, 78, 81, 83,
         85, 88]
index_2 = [1, 4, 25, 30, 72, 76]
index_3 = [26, 45, 58, 71, 73, 80]
for file in files[:1]:
    
    file_str = str(file)
    file_name = file_str.split("_")[0]
    print(file,file_name)
    os.makedirs(os.path.join(save_path,file_name), exist_ok=True)


    # os.makedirs((osp.join(save_path,file_name)), exist_ok=True)
    data_path = root  + file + '/' + file_name + '/T1w/Diffusion/data.nii.gz'
    bval_path = root  + file + '/' + file_name + '/T1w/Diffusion/bvals'
    bvec_path = root  + file + '/' + file_name + '/T1w/Diffusion/bvecs'
    # fa_path = root + file + '/' + file_name + '/T1w/Diffusion/fa.nii.gz'
    # ad_path = root + file + '/' + file_name + '/T1w/Diffusion/ad.nii.gz'
    # md_path = root + file + '/' + file_name + '/T1w/Diffusion/md.nii.gz'
    # od_path = noddi_path + file_name + '/FIT_OD.nii.gz'
    # ic_path = noddi_path + file_name + '/FIT_ICVF.nii.gz'
    # iso_path = noddi_path + file_name + '/FIT_ISOVF.nii.gz'
    mask_path = root  + file + '/' + file_name + '/T1w/Diffusion/nodif_brain_mask.nii.gz'
    
    
    rcnn_result_path = "/home/munan/VDT/rcnn/100206_inferred_90.nii.gz"
    rcnn,_=load_nifti(rcnn_result_path) 
    # dmri_out = f"/mnt/disk1/munan/hcp_single/100206/100206_inferred_sh.nii.gz"
    # sh,_ = load_nifti(dmri_out)
    
    # data.shape (145, 174, 145, 288)
    data, affine = load_nifti(data_path)
    mask,_ = load_nifti(mask_path)

    mask_new = nib.Nifti1Image(mask, np.eye(4))
    nib.save(mask_new,osp.join(save_path,file_name,f'{file_name}_brain_mask.nii.gz'))
    print("mask saved")
    
    # bvals = 0, 1000, 2000, 3000; bvecs.shape (288, 3)
    bvals, bvecs = read_bvals_bvecs(bval_path, bvec_path)
    # if data.shape[-1] != 288:
    #     raise "dwi original data's shape is not right"
    b0 = []
    b1 = []
    b2 = []
    b3 = []
    i = 0

    for i in range(288):
        b = bvals[i]
        if abs(b - 0) == min(abs(b - 0), abs(b - 1000), abs(b - 2000), abs(b - 3000)):
            b0.append(i)
        elif abs(b - 1000) == min(abs(b - 0), abs(b - 1000), abs(b - 2000), abs(b - 3000)):
            b1.append(i)
        elif abs(b - 2000) == min(abs(b - 0), abs(b - 1000), abs(b - 2000), abs(b - 3000)):
            b2.append(i)
        else:
            b3.append(i)
    b1_data = data[:, :, :, b1]
    print("data_b1: ",b1_data.min(), b1_data.max())
    b1_bvals = bvals[b1] 
    b1_bvecs = bvecs[b1, :]
    GT_mask, _ = load_nifti(mask_path)
    print("生成mask")
    if normalise:
        b0_data = data[:, :, :, b0] #shape [slice,h,w,dim]
        b0_bvals = bvals[b0]
        b0_bvecs = bvecs[b0, :]
        # print("GT mask shape:\t", GT_mask.shape)  
        average_b0 = np.mean(b0_data, axis=3) #shape [slice,h,w]
        b0 = np.expand_dims(average_b0, 3) # (145, 174, 145, 1)
        print("data_b0: ",b0.min(), b0.max())
        print("b0 average")          
        print("正则化begin")
        data_gt = b1_data / (b0+1e-5)#除b0正则化
        print("data_gt: ",data_gt.min(), data_gt.max())
        data_gt = np.nan_to_num(data_gt, copy=True, nan=0, posinf=0, neginf=0)
        data_gt = np.clip(data_gt, 0, 1)
        data_gt = data_gt * np.expand_dims(GT_mask, axis=-1)
        b1_data = data_gt
        print("正则化done")

        rcnn = rcnn/(b0+1e-5)
        rcnn = np.nan_to_num(rcnn, copy=True, nan=0, posinf=0, neginf=0)
        rcnn = np.clip(rcnn, 0, 1)
        rcnn = rcnn*np.expand_dims(GT_mask, axis=-1)
        new_image_norm = nib.Nifti1Image(rcnn, np.eye(4)) 
        nib.save(new_image_norm, osp.join("/home/munan/VDT/rcnn",f'100206_inferred_rcnn_2nd_norm.nii.gz'))
        # 
        # sh = sh/(b0+1e-5)
        # sh = np.nan_to_num(sh, copy=True, nan=0, posinf=0, neginf=0)
        # sh = np.clip(sh, 0, 1)
        # sh = sh*np.expand_dims(GT_mask, axis=-1)
        # new_image_norm_sh = nib.Nifti1Image(sh, np.eye(4)) 
        # nib.save(new_image_norm_sh, osp.join("/mnt/disk1/munan/hcp_single/100206/",f'100206_inferred_sh_norm.nii.gz'))
    
    else:
        b1_data = b1_data * np.expand_dims(GT_mask, axis=-1)
    print("data_b1: ",b1_data.min(), b1_data.max())
    
    # # print(b1_data.shape)
    # new_image = nib.Nifti1Image(b1_data, np.eye(4)) 
    # nib.save(new_image, osp.join(save_path,file_name,f'{file_name}_90.nii.gz'))
    # np.savetxt(osp.join(save_path,file_name,f'{file_name}_bvecs_90.txt'), b1_bvecs, delimiter=' ')
    # np.savetxt(osp.join(save_path,file_name,f'{file_name}_bvals_90.txt'), b1_bvals, delimiter=' ')

    # b1_data_selected = b1_data[:, :, :, index_1]
    # b1_bvals_selected = b1_bvals[index_1]
    # b1_bvecs_selected = b1_bvecs[index_1, :]
    # # print(b1_data_selected.shape)
    # new_image_6 = nib.Nifti1Image(b1_data_selected, np.eye(4)) 
    # nib.save(new_image_6, osp.join(save_path,file_name,f'{file_name}_06_2nd.nii.gz'))
    # np.savetxt(osp.join(save_path,file_name,f'{file_name}_bvecs_06_2nd.txt'), b1_bvecs_selected, delimiter=' ')
    # np.savetxt(osp.join(save_path,file_name,f'{file_name}_bvals_06_2nd.txt'), b1_bvals_selected, delimiter=' ')
    

    

In [13]:

#这是每个被试文件夹里有个小文件夹，装的当前被试的切片
import os
import numpy as np

# 基础路径
base_dir = "/mnt/siat205_disk5/munan/HCP_3T_251/"
# 零值占比阈值
# zero_ratio_threshold = 0.93

subject_dirs = [os.path.join(base_dir, d) for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]
subject_dirs = sorted(subject_dirs)
# 遍历所有子文件夹
for subject_dir in subject_dirs[210:220]:
    subject_id = os.path.basename(subject_dir)  # 提取编号
    file_path = os.path.join(subject_dir, "diffusion_90_directions", subject_id + "_90.nii.gz")
    output_dir_subject = os.path.join(subject_dir, "timehw")
    os.makedirs(output_dir_subject, exist_ok=True)
    
    if os.path.exists(file_path):
        # 加载数组
        array, _ = load_nifti(file_path)
        array = np.squeeze(array)  # shape [145, 174, 145, 90]
        # 转换操作
        # 1. 重新排列维度，相当于 permute(2, 3, 1, 0)
        transformed_array = np.transpose(array, (2, 3, 1, 0))  # [slice, direction, height, width] (145, 174, 145, 90)
        # 2. 翻转第三维，相当于 flip(dims=[2])
        transformed_array = np.flip(transformed_array, axis=2)

        # 遍历切片
        slice_count, direc_count, _, _ = transformed_array.shape
        for slice_idx in range(0,slice_count):
            # 获取切片
            sliced_image = transformed_array[slice_idx, :, :, :]  # shape [height, width]
            # 计算零值占比
            # zero_ratio = np.sum(sliced_image == 0) / sliced_image.size
            # if zero_ratio > zero_ratio_threshold:
            #     continue  # 跳过保存

            # 保存切片
            output_filename = f"sub{subject_id}_slice{slice_idx:03d}.npy"
            output_path = os.path.join(output_dir_subject, output_filename)
            np.save(output_path, sliced_image)

        print(f"处理完成: {subject_id}")
    else:
        print(f"文件不存在: {file_path}")


处理完成: 108525
处理完成: 124826
处理完成: 104012
处理完成: 102715
处理完成: 107018
处理完成: 112516
处理完成: 124220
处理完成: 125222
处理完成: 110411
处理完成: 111211
处理完成: 103818
处理完成: 110613
处理完成: 101107
处理完成: 108323
处理完成: 122317
处理完成: 123420
处理完成: 102311
处理完成: 120717
处理完成: 101410
处理完成: 101309
处理完成: 109830
处理完成: 112314
处理完成: 111009
处理完成: 103212
处理完成: 106016
处理完成: 122620
处理完成: 122418
处理完成: 103010
处理完成: 108121
处理完成: 121618
处理完成: 122822
处理完成: 105620
处理完成: 107321
处理完成: 100408
处理完成: 100307
处理完成: 107725
处理完成: 110007
处理完成: 121416
处理完成: 111413
处理完成: 105014
处理完成: 124624
处理完成: 121921
处理完成: 123723
处理完成: 123925
处理完成: 101006
处理完成: 101915
处理完成: 108828
处理完成: 121719
处理完成: 106319
处理完成: 100206
处理完成: 105923
处理完成: 102614
处理完成: 111716
处理完成: 103515
处理完成: 123521
处理完成: 106521
处理完成: 123117
处理完成: 105216
处理完成: 124422
处理完成: 104416
处理完成: 112112
处理完成: 102513
处理完成: 111312
处理完成: 103414
处理完成: 109123
处理完成: 111514
处理完成: 107422
处理完成: 102816
处理完成: 108020
处理完成: 104820
处理完成: 106824
处理完成: 100610
处理完成: 102109
处理完成: 112920
处理完成: 105115
处理完成: 108222
处理完成: 103111

In [6]:
import os
import shutil

source_dir = "/mnt/disk1/munan/subjects/"
files = [os.path.join(source_dir, f) for f in os.listdir(source_dir) if os.path.isdir(os.path.join(source_dir, f))]

for file in files:
    subject_id = file.split('_')[-1].split('.')[0]
    subject_folder = os.path.join(source_dir, subject_id)
    if not os.path.exists(subject_folder):
        os.makedirs(subject_folder)
    shutil.move(os.path.join(source_dir, file), os.path.join(subject_folder, file))
    
print("Files have been organized into subject folders.")


Files have been organized into subject folders.


In [3]:
root = '/mnt/disk4/hcp_fa_ad/hcp-78/'
save_path = '/mnt/disk1/munan/dwi/'
files = sorted(os.listdir(root))
dwi_90_allsubs = []
for file in files[:9]:
    print(file)
    file_str = str(file)
    file_name = file_str.split("_")[0]
    data_path = root + file + '/' + file_name + '/T1w/Diffusion/data.nii.gz'
    bval_path = root + file + '/' + file_name + '/T1w/Diffusion/bvals'
    bvec_path = root + file + '/' + file_name + '/T1w/Diffusion/bvecs'
    mask_path = root + file + '/' + file_name + '/T1w/Diffusion/nodif_brain_mask.nii.gz'

    data, affine = load_nifti(data_path)
    # bvals = 0, 1000, 2000, 3000; bvecs.shape (288, 3)
    bvals, bvecs = read_bvals_bvecs(bval_path, bvec_path)
    if data.shape[-1] != 288:
        print(data.shape)
        print(file_str)
        raise "dwi original data's shape is not right"
    #---------------------------------------获得各个shell的索引-----------------------------------------
    # 18个b0,其余每个b值各90  
    b0 = []
    b1 = []
    b2 = []
    b3 = []
    i = 0
    for i in range(288):
        b = bvals[i]
        if abs(b - 0) == min(abs(b - 0), abs(b - 1000), abs(b - 2000), abs(b - 3000)):
            b0.append(i)
            # print("b0:new index",i)
        elif abs(b - 1000) == min(abs(b - 0), abs(b - 1000), abs(b - 2000), abs(b - 3000)):
            b1.append(i)
            # print("b1:new index",i)
        elif abs(b - 2000) == min(abs(b - 0), abs(b - 1000), abs(b - 2000), abs(b - 3000)):
            b2.append(i)
        else:
            b3.append(i)
     #---------------------------------------------------------------------------------------------



100206_3T_Diffusion_preproc
100307_3T_Diffusion_preproc
100408_3T_Diffusion_preproc
100610_3T_Diffusion_preproc
101006_3T_Diffusion_preproc
101107_3T_Diffusion_preproc
101309_3T_Diffusion_preproc
101410_3T_Diffusion_preproc
101915_3T_Diffusion_preproc
102008_3T_Diffusion_preproc
(145, 174, 145, 191)
102008_3T_Diffusion_preproc


TypeError: exceptions must derive from BaseException

In [6]:
#这是一整个大文件夹里装的所有被试的切片
path = "/mnt/disk5/munan/HCP_3T_251/100206/diffusion_90_directions/100206_brain_mask.nii.gz"
# 基础路径
base_dir = "/mnt/disk5/munan/HCP_3T_251/"
output_dir = "/mnt/disk5/munan/sliced_data_251"  # 结果保存路径
os.listdir(base_dir)
os.makedirs(output_dir, exist_ok=True)  # 创建输出路径

# 零值占比阈值
zero_ratio_threshold = 0.93

subject_dirs = [os.path.join(base_dir, d) for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]

# 遍历所有子文件夹
for subject_dir in subject_dirs:
    subject_id = os.path.basename(subject_dir)  # 提取编号
    file_path = os.path.join(subject_dir,"diffusion_90_directions", subject_id + "_90.nii.gz")
    output_dir_subject = os.path.join(output_dir, subject_id)
    os.makedirs(output_dir_subject, exist_ok=True)
    
    if os.path.exists(file_path):
        # 加载nii,to numpy数组
        array, affine = load_nifti(file_path)
        array = np.squeeze(array)  # shape [145, 174, 145, 90]
        # 转换操作
        # 1. 重新排列维度，相当于 permute(2, 3, 1, 0)
        transformed_array = np.transpose(array, (2, 3, 1, 0))  # [slice, direction, height, width]
        # 2. 翻转第三维，相当于 flip(dims=[2])
        transformed_array = np.flip(transformed_array, axis=2)
        
        # 遍历切片
        slice_count, direc_count, _, _ = transformed_array.shape
        for slice_idx in range(10, slice_count - 10):
            # Check zero ratio across all directions for the current slice
            zero_ratios = [
                np.sum(transformed_array[slice_idx, direc_idx, :, :] == 0) /
                transformed_array[slice_idx, direc_idx, :, :].size
                for direc_idx in range(direc_count)
            ]
            
            # If any direction in the slice exceeds the zero ratio threshold, skip the entire slice
            if any(zero_ratio > zero_ratio_threshold for zero_ratio in zero_ratios):
                continue
            
            # Save all directions for the current slice
            for direc_idx in range(direc_count):
                sliced_image = transformed_array[slice_idx, direc_idx, :, :]  # shape [height, width]
                output_filename = f"sub{subject_id}_slice{slice_idx:03d}_direc{direc_idx:03d}.npy"
                output_path = os.path.join(output_dir_subject, output_filename)
                np.save(output_path, sliced_image)
        
        print(f"处理完成: {subject_id}")
    else:
        print(f"文件不存在: {file_path}")


GT mask shape:	 (145, 174, 145)
b0 average

b1_data_shape:	 (145, 174, 145, 90) 
b0_shape:	 (145, 174, 145, 1)


In [7]:
dwi_90_allsubs = []
progress_bar = tqdm(range(b1_data.shape[-1]))
for i in progress_bar:
    data_gt = b1_data[...,i:i+1] / (b0+1e-5)#除b0正则化

    data_gt = np.nan_to_num(data_gt, copy=True, nan=0, posinf=0, neginf=0)
    data_gt = np.clip(data_gt, 0, 1)
    data_gt = data_gt * np.expand_dims(GT_mask, axis=-1)

    dwi_90_allsubs.append(data_gt)
# data_gt_masked = np.concatenate(dwi_90_allsubs, axis=-1)
    # print("newly added sub'shape is: \t",dwi_90_allsubs[i].shape)


100%|██████████| 90/90 [00:03<00:00, 26.13it/s]


In [ ]:
##------------generate /munan/subjects,每一个sub的90方向volume生成-------##
root = '/mnt/disk4/hcp_fa_ad/hcp-78/'
save_path = '/mnt/disk1/munan/subjects/'
files = sorted(os.listdir(root))
files.pop(9)
os.makedirs(save_path, exist_ok=True)
index_1 = [7, 37, 39, 68, 69, 81]
index_2 = [1, 4, 25, 30, 72, 76]
index_3 = [26, 45, 58, 71, 73, 80]
prog_bar = tqdm(enumerate(files[:]))
for i, file in prog_bar:

    file_str = str(file)
    file_name = file_str.split("_")[0]
    print(i,file,file_name)
    # os.makedirs((osp.join(save_path,file_name)), exist_ok=True)
    # data_path = root  + file_name + '/dmri.nii.gz'
    bval_path = root + file + '/'+ file_name + '/T1w/Diffusion/bvals'
    bvec_path = root + file + '/'+ file_name + '/T1w/Diffusion/bvecs'
    # fa_path = root + file + '/' + file_name + '/T1w/Diffusion/fa.nii.gz'
    # ad_path = root + file + '/' + file_name + '/T1w/Diffusion/ad.nii.gz'
    # md_path = root + file + '/' + file_name + '/T1w/Diffusion/md.nii.gz'
    # od_path = noddi_path + file_name + '/FIT_OD.nii.gz'
    # ic_path = noddi_path + file_name + '/FIT_ICVF.nii.gz'
    # iso_path = noddi_path + file_name + '/FIT_ISOVF.nii.gz'
    mask_path = root + file + '/'+ file_name + '/T1w/Diffusion/nodif_brain_mask.nii.gz'
    # data.shape (145, 174, 145, 288)
    # data, affine = load_nifti(data_path)
    mask,_ = load_nifti(mask_path)
    np.save(os.path.join(save_path,f'brain_mask_{file_name}.npy'),mask)
    # mask_new = nib.Nifti1Image(mask, np.eye(4))
    # nib.save(mask_new,osp.join(save_path,file_name,f'brain_mask_{file_name}.nii.gz'))
    print("mask saved")
    # bvals = 0, 1000, 2000, 3000; bvecs.shape (288, 3)
    bvals, bvecs = read_bvals_bvecs(bval_path, bvec_path)
    # if data.shape[-1] != 288:
    #     raise "dwi original data's shape is not right"
    b0 = []
    b1 = []
    b2 = []
    b3 = []
    i = 0
    for i in range(288):
        b = bvals[i]
        if abs(b - 0) == min(abs(b - 0), abs(b - 1000), abs(b - 2000), abs(b - 3000)):
            b0.append(i)
        elif abs(b - 1000) == min(abs(b - 0), abs(b - 1000), abs(b - 2000), abs(b - 3000)):
            b1.append(i)
        elif abs(b - 2000) == min(abs(b - 0), abs(b - 1000), abs(b - 2000), abs(b - 3000)):
            b2.append(i)
        else:
            b3.append(i)
    # b1_data = data[:, :, :, b1]
    b1_bvals = bvals[b1] 
    b1_bvecs = bvecs[b1, :]
    # print(b1_data.shape)
    # new_image = nib.Nifti1Image(b1_data, np.eye(4)) 
    # nib.save(new_image, osp.join(save_path,file_name,f'{file_name}_90.nii.gz'))
    np.savetxt(osp.join(save_path,f'bvecs_90_{file_name}.txt'), b1_bvecs, delimiter=' ')
    # np.savetxt(osp.join(save_path,f'bvals_90_{file_name}.txt'), b1_bvals, delimiter=' ')

    # b1_data_selected = b1_data[:, :, :, index_1]
    # b1_bvals_selected = b1_bvals[index_1]
    # b1_bvecs_selected = b1_bvecs[index_1, :]
    # print(b1_data_selected.shape)
    # new_image_6 = nib.Nifti1Image(b1_data_selected, np.eye(4)) 
    # nib.save(new_image_6, osp.join(save_path,file_name,f'{file_name}_06.nii.gz'))
    # np.savetxt(osp.join(save_path,file_name,f'{file_name}_bvecs_06.txt'), b1_bvecs_selected, delimiter=' ')
    # np.savetxt(osp.join(save_path,file_name,f'{file_name}_bvals_06.txt'), b1_bvals_selected, delimiter=' ')